%md
<img src="https://raw.githubusercontent.com/Databricks-BR/health/main/image/head_notebook.png" width="1000px">

<img src="https://raw.githubusercontent.com/ArcInstitute/evo2/refs/heads/main/evo2.jpg" width="500px">



##### Descrição e Objetivos

Este projeto explora o uso de modelos avançados de inteligência artificial para análise e predição de sequências de DNA e proteínas. O objetivo principal é demonstrar a integração de tecnologias modernas como o **evo2-40b** (da NVIDIA, um modelo de fundação biológica capaz de integrar informações ao longo de longas sequências genômicas, mantendo sensibilidade a alterações de nucleotídeos individuais e gerando uma potencial sequência de DNA), o **esmfold** (da NVIDIA, para predição da estrutura 3D de proteínas a partir de sequências de aminoácidos) e ferramentas de **visualização 3D** como graphein e plotly.

##### Referências

* [Evo2 Blog](https://arcinstitute.org/news/blog/evo2)
* [GitHub - Evo2](https://github.com/ArcInstitute/evo2)
* [Evo2-40b API](https://build.nvidia.com/arc/evo2-40b?snippet_tab=Try)
* [esmfold API](https://build.nvidia.com/meta/esmfold?snippet_tab=Python)
* [Trabalhando com arquivos .pdb em Python](https://medium.com/@jgbrasier/working-with-pdb-files-in-python-7b538ee1b5e4)
* https://developer.nvidia.com/blog/accelerating-drug-discovery-at-receptor-ai-with-nvidia-bionemo-cloud-apis
* https://github.com/NVIDIA/bionemo-framework
* https://github.com/NVIDIA/bionemo-examples/tree/main/examples/nims
* https://github.com/NVIDIA/bionemo-examples/blob/main/examples/nims/alphafold2/AlphaFold2-NIM-example.ipynb
* https://developer.nvidia.com/blog/understanding-the-language-of-lifes-biomolecules-across-evolution-at-a-new-scale-with-evo-2/

##### Controle de Versão do Código

| versão | data | autor | e-mail | alterações |
| --- | --- | --- | --- | --- |
| 1.0 | 15-JUN-2025 | Bruna Robledo<br>Luis Assunção<br>Vinicius Fialho | bruna.robledo@databricks.com<br>luis.assuncao@databricks.com<br>vinicius.fialho@databricks.com | Primeira versão  |

##### Descrição do Cluster

Toda a demo pode ser executada utilizando Serverless ou um tipo de instância de sua pereferência.

In [0]:
import requests
import os
import json
from pathlib import Path

### Gera uma potencial sequência de DNA

Para conseguir uma chave para testar a API, entre no [site da NVIDIA](https://build.nvidia.com/), crie a sua conta e faça login. Após isso, clique em Get API Key e salve o valor, cole no campo "key" na célula a seguir.

Para acessar a página específica do Evo2, entre [aqui](https://build.nvidia.com/arc/evo2-40b).

In [0]:
key = "INSIRA-AQUI-A-SUA-CHAVE-NVIDIA-PARA-TESTAR"
sequence_input = "GAATAGGAACAGCTCCGGTCTACAGCTCCCAGCGTGAGCGACGCAGAAGACGGTGATTTCTGCATTTCCATCTGAGGTACCGGGTTCATCTCACTAGGGAGTGCCAGACAGTGGGCGCAGGCCAGTGTGTGTGCGCACCGTGCGCGAGCCGAAGCAGGGCGAGGCATTGCCTCACCTGGGAAGCGCAAGGGGTCAGGGAGTTCCCTTTCCGAGTCAAAGAAAGGGGTGATGGACGCACCTGGAAAATCGGGTCACTCCCACCCGAATATTGCGCTTTTCAGACCGGCTTAAGAAACGGCGCACCACGAGACTATATCCCACACCTGGCTCAGAGGGTCCTACGCCCACGGAATC"

r = requests.post(
    url=os.getenv(
        "URL", "https://health.api.nvidia.com/v1/biology/arc/evo2-40b/generate"
    ),
    headers={"Authorization": f"Bearer {key}"},  # Chave da API para consulta
    json={
        # Definir a sequência de DNA em que a predição será feita
        "sequence": sequence_input,
        "num_tokens": 102,  # Quantidade de tokens que queremos que seja gerada a partir da sequência fornecida
        "top_k": 4,  # Quantidade de tokens com maior probabilidade serão considerados na geração do modelo; escolhe entre as K melhores opções
        "top_p": 1,  # Controla a diversidade das previsões considerando apenas os tokens mais prováveis até atingir uma probabilidade acumulada especificada, com top_p = 1 todos os tokens são considerados sem nenhuma limitação
        "temperature": 0.7,  # Controla a aleatoriedade das escolhas do modelo. Valores baixos geram respostas mais previsíveis; valores altos tornam as respostas mais variadas e criativas
        "enable_sampled_probs": True,
    },
)

if "application/json" in r.headers.get("Content-Type", ""):
    print(r, "Saving to output.json:\n", r.text[:200], "...")
    Path("output.json").write_text(r.text)
elif "application/zip" in r.headers.get("Content-Type", ""):
    print(r, "Saving large response to data.zip")
    Path("data.zip").write_bytes(r.content)
else:
    print(r, r.headers, r.content)

### Visualize a sequência fornecida junto da sequência gerada

In [0]:
import plotly.graph_objects as go

# Lê o arquivo JSON
with open("output.json") as f:
    output_data = json.load(f)

# Extrai a sequência de DNA e as probabilidades a partir da previsão gerada
sequence = output_data["sequence"]
probabilities = output_data["sampled_probs"]

# Configuração
LINE_LENGTH = 30  # Caracteres por linha - ajustar conforme necessário
NUCLEOTIDE_OFFSET = 0.08  # Valor positivo para colocar as letras acima da linha
FONT_SIZE = 12

# Define o valor hexadecimal das cores
HEX_GREEN = "#2DC468"
HEX_BLUE = "#2D68C4"
HEX_RED = "#C42D2D"

# Combina a sequência de DNA de input com a sequência gerada
full_sequence = sequence_input + sequence
full_probabilities = [None] * len(sequence_input) + probabilities

# Divide a sequência em linhas
lines = []
start = 0
while start < len(full_sequence):
    end = min(start + LINE_LENGTH, len(full_sequence))
    lines.append((full_sequence[start:end], full_probabilities[start:end], start))
    start = end

fig = go.Figure()

# Plota cada linha
for line_idx, (line_seq, line_probs, offset) in enumerate(lines):
    y_offset = -line_idx * 0.5  # Cada nova linha vai abaixo da anterior

    # Plota os segmentos da linha
    for i in range(len(line_seq) - 1):
        if line_probs[i] is None and line_probs[i + 1] is None:
            # Coloca a sequência de input sempre com a cor verde
            color = HEX_GREEN
            hoverinfo = "skip"
            text = ""
        elif line_probs[i] is None or line_probs[i + 1] is None:
            # Faz a transição entre a sequência de input e a sequência gerada
            if line_probs[i] is None:
                avg_prob = line_probs[i + 1]
            else:
                avg_prob = line_probs[i]
            color = HEX_BLUE if avg_prob >= 0.4 else HEX_RED
            hoverinfo = "text"
            text = f"Indexes: {offset+i}-{offset+i+1}<br>Prob: {avg_prob:.2f}"
        else:
            avg_prob = (line_probs[i] + line_probs[i + 1]) / 2
            color = HEX_BLUE if avg_prob >= 0.4 else HEX_RED
            hoverinfo = "text"
            text = f"Indexes: {offset+i}-{offset+i+1}<br>Probs: {line_probs[i]:.2f}, {line_probs[i+1]:.2f}"

        fig.add_trace(
            go.Scatter(
                x=[i, i + 1],
                y=[y_offset, y_offset],
                mode="lines",
                line=dict(color=color, width=16),
                hoverinfo=hoverinfo,
                text=text,
                showlegend=False,
            )
        )

    # Coloca os valores da sequência de DNA acina de cada linha
    for i, nuc in enumerate(line_seq):
        if i < len(line_probs):
            if line_probs[i] is None:
                letter_color = HEX_GREEN  # Verde para sequence_input
            else:
                letter_color = HEX_BLUE if line_probs[i] >= 0.4 else HEX_RED
        else:
            letter_color = HEX_GREEN  # Verde como padrão caso não tenha probabilidade

        fig.add_annotation(
            x=i,
            y=y_offset + NUCLEOTIDE_OFFSET,
            text=nuc,
            showarrow=False,
            font=dict(size=FONT_SIZE, color=letter_color),
            yanchor="bottom",
        )

# Ajustes finais de layout
fig.update_layout(
    title={
        "text": "Sequência de DNA com Probabilidade da Previsão",
        "x": 0.5,
        "xanchor": "center",
    },
    xaxis=dict(
        tickmode="linear",
        tick0=0,
        dtick=1,
        showgrid=False,
        range=[-0.5, LINE_LENGTH - 0.5],
        showticklabels=False,
    ),
    yaxis=dict(visible=False),
    hovermode="closest",
    margin=dict(t=60, b=40),
    plot_bgcolor="white",
    height=200 + 50 * (len(lines) - 1),
)

fig.show()

### Utilize o esmfold (NVIDIA) para predizer a estrutura 3D
O esmfold prediz a estrutura 3D de uma proteína a partir da sua sequência de aminoácidos.

In [0]:
# Transforma a sequência em um formato .json
invoke_url = "https://health.api.nvidia.com/v1/biology/nvidia/esmfold"

headers = {
    "Authorization": f"Bearer {key}",
    "Accept": "application/json",
}

payload = {
    "sequence": full_sequence
}

# re-use connections
session = requests.Session()

response = session.post(invoke_url, headers=headers, json=payload)

response.raise_for_status()
response_body = response.json()
print(response_body)

### Transforme toda a sequência em um arquivo .pdb

Um arquivo PDB (Protein Data Bank) é um formato de arquivo padronizado baseado em texto que armazena dados estruturais tridimensionais (3D) de macromoléculas biológicas, principalmente proteínas e ácidos nucleicos, bem como seus complexos. Esses arquivos contêm as coordenadas atômicas, informações de ligações e outros detalhes relevantes das biomoléculas derivados de métodos experimentais.

Com isso, conseguiremos utilizar bibliotecas para visualização 3D de toda a sequência gerada.

In [0]:
# Extraindo o texto PDB (pegando o primeiro item da lista)
pdb_text = response_body["pdbs"][0]

# Caminho para salvar o arquivo PDB
output_dir = Path.cwd()
output_path = os.path.join(output_dir, "protein_structure.pdb")

# Criando o diretório se não existir
os.makedirs(output_dir, exist_ok=True)

# Salvando o arquivo PDB
with open(output_path, "w") as f:
    f.write(pdb_text)

print(f"Arquivo PDB salvo em: {output_path}")

## Gerando visualizações 3D com Plotly e Graphein

### Plotly

In [0]:
%pip install biopandas prody --upgrade

In [0]:
import pandas as pd
from biopandas.pdb import PandasPdb
from prody import parsePDBHeader
from typing import Optional


def read_pdb_to_dataframe(
    pdb_path: Optional[str] = None,
    model_index: int = 1,
    parse_header: bool = True,
) -> pd.DataFrame:
    """
    Read a PDB file, and return a Pandas DataFrame containing the atomic coordinates and metadata.

    Args:
        pdb_path (str, optional): Path to a local PDB file to read. Defaults to None.
        model_index (int, optional): Index of the model to extract from the PDB file, in case
            it contains multiple models. Defaults to 1.
        parse_header (bool, optional): Whether to parse the PDB header and extract metadata.
            Defaults to True.

    Returns:
        pd.DataFrame: A DataFrame containing the atomic coordinates and metadata, with one row
            per atom
    """
    atomic_df = PandasPdb().read_pdb(pdb_path)
    if parse_header:
        header = parsePDBHeader(pdb_path)
    else:
        header = None
    atomic_df = atomic_df.get_model(model_index)
    if len(atomic_df.df["ATOM"]) == 0:
        raise ValueError(f"No model found for index: {model_index}")

    return pd.concat([atomic_df.df["ATOM"], atomic_df.df["HETATM"]]), header

In [0]:
df, df_header = read_pdb_to_dataframe("protein_structure.pdb")
df.head(10)

In [0]:
%pip install pandas==2.3.0 --upgrade
%pip install --upgrade plotly

In [0]:
import plotly.express as px

fig = px.scatter_3d(df, x="x_coord", y="y_coord", z="z_coord", color="element_symbol")
fig.update_traces(marker_size=4)

fig.show()

### Graphein

In [0]:
%pip install graphein
%pip install biopandas prody --upgrade

dbutils.library.restartPython()  # Restart Python to apply changes

In [0]:
import pandas as pd
from biopandas.pdb import PandasPdb
from prody import parsePDBHeader
from typing import Optional


def read_pdb_to_dataframe(
    pdb_path: Optional[str] = None,
    model_index: int = 1,
    parse_header: bool = True,
) -> pd.DataFrame:
    """
    Read a PDB file, and return a Pandas DataFrame containing the atomic coordinates and metadata.

    Args:
        pdb_path (str, optional): Path to a local PDB file to read. Defaults to None.
        model_index (int, optional): Index of the model to extract from the PDB file, in case
            it contains multiple models. Defaults to 1.
        parse_header (bool, optional): Whether to parse the PDB header and extract metadata.
            Defaults to True.

    Returns:
        pd.DataFrame: A DataFrame containing the atomic coordinates and metadata, with one row
            per atom
    """
    atomic_df = PandasPdb().read_pdb(pdb_path)
    if parse_header:
        header = parsePDBHeader(pdb_path)
    else:
        header = None
    atomic_df = atomic_df.get_model(model_index)
    if len(atomic_df.df["ATOM"]) == 0:
        raise ValueError(f"No model found for index: {model_index}")

    return pd.concat([atomic_df.df["ATOM"], atomic_df.df["HETATM"]]), header

In [0]:
df, df_header = read_pdb_to_dataframe("protein_structure.pdb")
df.head(10)

In [0]:
from graphein.protein.graphs import label_node_id


def process_dataframe(df: pd.DataFrame, granularity="CA") -> pd.DataFrame:
    """
    Process a DataFrame of protein structure data to reduce ambiguity and simplify analysis.

    This function performs the following steps:
    1. Handles alternate locations for an atom, defaulting to keep the first one if multiple exist.
    2. Assigns a unique node_id to each residue in the DataFrame, using a helper function label_node_id.
    3. Filters the DataFrame based on specified granularity (defaults to 'CA' for alpha carbon).

    Parameters
    ----------
    df : pd.DataFrame
        The DataFrame containing protein structure data to process. It is expected to contain columns 'alt_loc' and 'atom_name'.

    granularity : str, optional
        The level of detail or perspective at which the DataFrame should be analyzed. Defaults to 'CA' (alpha carbon).
    """
    # handle the case of alternative locations,
    # if so default to the 1st one = A
    if "alt_loc" in df.columns:
        df["alt_loc"] = df["alt_loc"].replace("", "A")
        df = df.loc[(df["alt_loc"] == "A")]
    df = label_node_id(df, granularity)
    df = df.loc[(df["atom_name"] == granularity)]
    return df

In [0]:
process_df = process_dataframe(df)
print(process_df.shape)

In [0]:
from graphein.protein.graphs import initialise_graph_with_metadata

g = initialise_graph_with_metadata(
    protein_df=process_df,
    raw_pdb_df=df,
    pdb_code="3nir",
    granularity="CA",
)

In [0]:
from graphein.protein.graphs import add_nodes_to_graph

g = add_nodes_to_graph(g)
print(g.nodes)

In [0]:
import networkx as nx


def add_backbone_edges(G: nx.Graph) -> nx.Graph:
    # Iterate over every chain
    for chain_id in G.graph["chain_ids"]:
        # Find chain residues
        chain_residues = [
            (n, v) for n, v in G.nodes(data=True) if v["chain_id"] == chain_id
        ]
        # Iterate over every residue in chain
        for i, residue in enumerate(chain_residues):
            try:
                # Checks not at chain terminus
                if i == len(chain_residues) - 1:
                    continue
                # Asserts residues are on the same chain
                cond_1 = residue[1]["chain_id"] == chain_residues[i + 1][1]["chain_id"]
                # Asserts residue numbers are adjacent
                cond_2 = (
                    abs(
                        residue[1]["residue_number"]
                        - chain_residues[i + 1][1]["residue_number"]
                    )
                    == 1
                )

                # If this checks out, we add a peptide bond
                if (cond_1) and (cond_2):
                    # Adds "peptide bond" between current residue and the next
                    if G.has_edge(i, i + 1):
                        G.edges[i, i + 1]["kind"].add("backbone_bond")
                    else:
                        G.add_edge(
                            residue[0],
                            chain_residues[i + 1][0],
                            kind={"backbone_bond"},
                        )
            except IndexError as e:
                print(e)
    return G

In [0]:
g = add_backbone_edges(g)
print(len(g.edges()))

In [0]:
from graphein.protein.visualisation import plotly_protein_structure_graph

p = plotly_protein_structure_graph(
    g,
    colour_edges_by="kind",
    colour_nodes_by="seq_position",
    label_node_ids=False,
    plot_title="3NIR Backbone Protein Graph",
    node_size_multiplier=1,
)
p.show()